In [ ]:
from cProfile import label
import sys
import json
import math
import os
import ast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt

import os
import seaborn as sns

# `Metrics at Episode Level`

The following cells sorth through all the episodes at the team leavel. First they plot the metrics for every episode at team level. Then plot a correlation plote metric vs score.

It plots episode 1 vs episode 2 or high vs low score at the global level (averaged).s

In [ ]:
plt.ioff()

directory ='/Users/josecordova/Desktop/Michigan/545/project/TED-Minimap/output/'# '/Users/josecordova/Documents/cmu/TED'

scores=[]
skills=[]
efforts=[]
workloads=[]
labels = []
CIs = []

os.chdir(directory)
metrics=['Effort','Skill','Workload']
df1 =pd.DataFrame(0, index=np.arange(29), columns=metrics)
df2 =pd.DataFrame(0, index=np.arange(29), columns=metrics)

num_files_ep1 = 0
num_files_ep2 = 0

file_end = 'High vs low score1' # 'Ep1 vs Ep2 # File termination to save plots according to the filter

for name in sorted(os.listdir(directory)):

    if not name.endswith(".csv") : 
        continue

    df= pd.read_csv(name)

    df['score']=50*df['triage_count_red']+df['triage_count_yellow']*30+df['triage_count_green']*10
    curr_score=df['score'].sum()
    if curr_score >300:#'1' in name[21:24]:   # filter for episode, change commented expression according to high vs low score or ep1 vs ep2
        df1['Effort'] +=df['process_effort_s']
        df1['Skill'] +=df['process_skill_use_s']
        df1['Workload'] +=df['process_workload_burnt']
        num_files_ep1 +=1

        #continue
    
    elif curr_score<100: # Change to else if trying to get episode 1 vs ep2
        df2['Effort'] +=df['process_effort_s']
        df2['Skill'] +=df['process_skill_use_s']
        df2['Workload'] +=df['process_workload_burnt']
        num_files_ep2 +=1

    scores.append(curr_score)

    labels.append(name[11:-11])
    df['time'] = range(0,10*len(df),10)
    workloads.append(df['process_workload_burnt'].sum())
    skills.append(df['process_skill_use_s'].sum())
    efforts.append(df['process_effort_s'].sum())

    df['CI']=df['process_workload_burnt']+df['process_skill_use_s']+df['process_effort_s']
    CIs.append(df['CI'].sum())

    df.rename(columns = {'process_skill_use_s':'Skill', 'process_effort_s':'Effort','process_workload_burnt':'Workload'}, inplace = True)

    plot_labels = ['Workload','Skill','Effort','CI']

    for proccess in plot_labels:
        plt.figure()
        df.plot(y=proccess,x='time')
        plt.xlabel('Time(s)')
        plt.ylabel(proccess)
        plt.title(proccess)
        plt.ylim(top=1.5,bottom=0)
        plt.savefig(name+proccess+'1.png',facecolor='w')
        plt.close()



## Global Scatter Plots

In [ ]:


colors = [ 'b' , 'c', 'g', 'y', 'm', 'k', 'r', 'peru', 'silver' ]
plot_labels = [skills, efforts, workloads, CIs]
fig_names=['Skill', 'Effort', 'Workload', 'CI']

for i, label in enumerate(plot_labels):
  sns.scatterplot(x=label, y=scores, hue=labels)
  plt.xlabel(fig_names[i])
  plt.ylabel('Score')
  plt.title(fig_names[i]+' Correlation')
  lgd=plt.legend(loc=(1.04,0), prop={'size': 6},ncol=3)
  plt.savefig(fig_names[i]+'1.png', bbox_extra_artists=(lgd,), bbox_inches='tight',facecolor='w')


## High vs Low Score or Ep1 vs Ep2

1.   List item
2.   List item



In [ ]:


df2 =df2/num_files_ep2
df1 =df1/num_files_ep1
lims=[1,0.2,0.5]

for i, metric in enumerate(metrics):
    curr=pd.DataFrame()
    curr['High']=df1[metric]
    curr['Low']=df2[metric]
    curr['time'] = range(0,10*len(df1),10)

    plt.figure()
    curr.plot(x='time')

    plt.xlabel('Time(s)')
    plt.ylabel(metric)
    plt.title(metric)
    plt.ylim(top=lims[i])

    plt.savefig(directory+file_end+metric+'1.png',facecolor='w')
    plt.close()


# Role level graphs

Graphs the metrics at role level (med vs engineer) within episode and globaly (or filter for number of episode or score).




In [ ]:
%reset

In [ ]:
from cProfile import label
import sys
import json
import math
import os
import ast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt

import os
import seaborn as sns

In [ ]:


directory ='/Users/josecordova/Desktop/Michigan/545/project/TED-Minimap/output/'# '/Users/josecordova/Documents/cmu/TED'

metrics=['Effort','Skill','Workload']
df_eng =pd.DataFrame(0, index=np.arange(25), columns=metrics)
df_med =pd.DataFrame(0, index=np.arange(25), columns=metrics)
num_meds=0
num_engs=0
plt.ioff()
for name_folder in os.listdir(directory):
    if '.' in name_folder: 
        continue
    i=1
    directory_2 = directory + name_folder
    curr_global = pd.read_csv(directory_2+'_global.csv')
    curr_score = 50*curr_global['triage_count_red']+curr_global['triage_count_yellow']*30+curr_global['triage_count_green']*10
    curr_score=curr_score.sum()
    flag=1
    #if curr_score<300:
     #   continue
    for name in os.listdir(directory_2):

        if not name.endswith(".csv") : 
            continue


        df = pd.read_csv(directory_2+'/'+name)
        if flag==1:
            df_effort=pd.DataFrame(0, index=np.arange(len(df)), columns=['Med','Eng'])
            df_triage=pd.DataFrame(0, index=np.arange(len(df)), columns=['Med','Eng'])
            df_speedup=pd.DataFrame(0, index=np.arange(len(df)), columns=['Med','Eng'])
            df_rubble=pd.DataFrame(0, index=np.arange(len(df)), columns=['Med','Eng'])
            df_exploration=pd.DataFrame(0, index=np.arange(len(df)), columns=['Med','Eng'])
            df_workload=pd.DataFrame(0, index=np.arange(len(df)), columns=['Med','Eng'])
            df_skill=pd.DataFrame(0, index=np.arange(len(df)), columns=['Med','Eng'])
            flag=0


        curr_player = 'player ' + str(i)

        if name[0]=='e':
            curr= 'Eng'
            df_eng['Workload']+=df['workload']
            df_eng['Skill']+=df['skill_s']
            df_eng['Effort']+=df['effort']
            num_engs+=1


        else:
            curr='Med'
            df_med['Workload']+=df['workload']
            df_med['Skill']+=df['skill_s']
            df_med['Effort']+=df['effort']
            num_meds+=1

        df_workload[curr]+=df['workload']
        df_skill[curr]+=df['skill_s']
        df_effort[curr]+=df['effort']
        '''
        df_exploration[curr]+=df['exploration']

        df_triage[curr]+=df['action_triage_s']
        df_speedup[curr]+=df['action_speedup_s'] 
        df_rubble[curr]+=df['action_dig_rubble_s']'''


        i+=1
    df_effort['time'] = range(0,10*len(df),10)
    df_effort[df_effort<0]=0
    df_effort.plot(x='time')
    plt.xlabel('Time(s)')
    plt.ylabel('Effort')
    plt.title('Effort')
    plt.savefig(directory_2+'/effort_roles1.png',facecolor='w')


    df_workload['time'] = range(0,10*len(df),10)

    df_workload.plot(x='time')
    plt.xlabel('Time(s)')
    plt.ylabel('Workload')
    plt.title('Workload')

    plt.savefig(directory_2+'/workload_roles1.png',facecolor='w')
    df_skill['time'] = range(0,10*len(df),10)

    df_skill.plot(x='time')
    plt.xlabel('Time(s)')
    plt.ylabel('Skill (s)')
    plt.title('Skill')

    plt.savefig(directory_2+'/skill_roles1.png',facecolor='w',facecolor='w')


df_eng['time'] = range(0,10*len(df_eng),10)
df_med['time'] = range(0,10*len(df_eng),10)

df_med=df_med/num_meds
df_eng=df_eng/num_engs

lims=[1,0.2,0.5]
i=0
for metric in metrics:
    curr=pd.DataFrame()
    curr['Medic']=df_med[metric]
    curr['Eng.']=df_eng[metric]
    curr['time'] = range(0,10*len(df_eng),10)

    plt.figure()
    curr.plot(x='time')

    plt.xlabel('Time(s)')
    plt.ylabel(metric)
    plt.title(metric )
    plt.ylim(top=lims[i])

    plt.savefig(directory+'Med vs Eng '+metric+'1.png',facecolor='w')
    plt.close()
    i+=1
